# AC 221 - Problem Set 3

## Problem 1

### Finding Duplicates

In [5]:
import numpy as np
import csv

We begin by reading in the csv with the `csv` Python functionality and using a modified version of [Jim's `find_dups` function](https://github.com/jimwaldo/APCOMP221Code/blob/master/find_duplicates.py) to find duplicates. 

#### Useful Functions

In [70]:
def get_key(x):
    return (x['course_id'], x['user_id'])
    
# Jim's Function (Modified)
"""
Created on 2019-03-14
@author waldo
"""

def find_dups(csv_in, csv_out):
    total_lines = 0
    unique_lines = 0
    dup_lines = 0
    lines_seen = set()

    for l in csv_in:
        total_lines += 1
        key = get_key(l)
        if key not in lines_seen:
            lines_seen.add(key)
            csv_out.writerow(l)
            unique_lines += 1
        else:
            dup_lines += 1
    return total_lines, unique_lines, dup_lines

In [71]:
# Open original csv to get headers
with open("sample_set.csv", "r") as org_csv:
    reader = csv.reader(org_csv)
    headers = next(reader)

In [72]:
print(headers)

['course_id', 'user_id', 'registered', 'viewed', 'explored', 'certified', 'completed', 'ip', 'cc_by_ip', 'countryLabel', 'continent', 'city', 'region', 'subdivision', 'postalCode', 'un_major_region', 'un_economic_group', 'un_developing_nation', 'un_special_region', 'latitude', 'longitude', 'LoE', 'YoB', 'gender', 'grade', 'passing_grade', 'start_time', 'first_event', 'last_event', 'nevents', 'ndays_act', 'nplay_video', 'nchapters', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 'roles', 'nprogcheck', 'nproblem_check', 'nforum_events', 'mode', 'is_active', 'cert_created_date', 'cert_modified_date', 'cert_status', 'verified_enroll_time', 'verified_unenroll_time', 'profile_country', 'y1_anomalous', 'email_domain', 'language_brwsr', 'language_brwsr_country', 'language_brwsr_sec', 'language_brwsr_sec_country', 'language_brwsr_code', 'language_brwsr_subcode', 'language_brwsr_sec_code', 'language_brwsr_sec_subcode', 'language_brwsr_nev

In [73]:
# Run function
with open("sample_set.csv", "r", encoding = "utf8") as org_file:
    org_csv = csv.DictReader(org_file)
    with open("sample_set_unique.csv", "w", encoding = "utf8") as unique_file:
        unique_csv = csv.DictWriter(unique_file, fieldnames = headers)
        
        total_lines, unique_lines, dup_lines = find_dups(org_csv, unique_csv)

In [74]:
print("Total lines: {}".format(total_lines))
print("Unique lines: {}".format(unique_lines))
print("Duplicated lines: {}".format(dup_lines))

Total lines: 1100005
Unique lines: 137450
Duplicated lines: 962555


### Repairing Values

We will be looking at all original records for repairing fields. For example, if a duplicate records exist for a student/class pair, we can check if the duplicates have some of the bad or missing values. 

In [ ]:
# Read and store original records
original_records = []

with open("sample_set.csv", "r", encoding = "utf8") as org_file:
    org_csv = csv.DictReader(org_file)
    for row in org_csv:
        original_records.append(row)

#### Useful Functions

In [113]:
# Getting all unique values in a column and corresponding frequency
def column_frequencies(theRecords, theColumn):
    frequency_dict = {}
    for record in theRecords:
        record_column = record[theColumn]
        if record_column in frequency_dict:
            frequency_dict[record_column] += 1
        else:
            frequency_dict[record_column] = 1
    return frequency_dict

def print_sorted_col_by_freq(frequencyDict):
    print({k: v for k, v in sorted(frequencyDict.items(), key=lambda item: item[1])})
    
def print_sorted_col_by_key(frequencyDict):
    for key in sorted(frequencyDict.keys()):
        print(key, frequencyDict[key])

#### Find Bad or Missing Values

In [122]:
# Take a look at columns by general categorical group
registration_cols = ['registered', 'viewed', 'explored', 'certified', 'completed']
location_cols = ['cc_by_ip', 'countryLabel', 'continent', 'city', 'region', 'subdivision', 'postalCode', 'latitude', 'longitude']
un_cols = ['un_major_region', 'un_economic_group', 'un_developing_nation', 'un_special_region']
demo_cols = ['LoE', 'YoB', 'gender']
grade_cols = ['grade', 'passing_grade']
time_date_cols = ['start_time', 'first_event', 'last_event', 'nevents'] 
action_cols = ['ndays_act', 'nplay_video', 'nchapters'] 
forum_cols = ['nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned' 
               'roles', 'nprogcheck', 'nproblem_check', 'nforum_events'] 
cert_cols = ['cert_created_date', 'cert_modified_date', 'cert_status']
enroll_cols = ['verified_enroll_time', 'verified_unenroll_time']
language_cols = ['language_brwsr', 'language_brwsr_country', 'language_brwsr_sec', 'language_brwsr_sec_country', 
                 'language_brwsr_code', 'language_brwsr_subcode', 'language_brwsr_sec_code', 'language_brwsr_sec_subcode', 
                 'language_brwsr_nevents', 'language_brwsr_ndiff', 
                 'language', 'language_download', 'language_nevents', 'language_ndiff']
video_cols = ['ntranscript', 'nshow_answer', 'nvideo', 'nvideos_unique_viewed', 'nvideos_total_watched', 'nseq_goto', 
              'nseek_video', 'npause_video']
dt_cols = ['avg_dt', 'sdv_dt', 'max_dt', 'n_dt', 'sum_dt']
roles_cols = ['roles_isBetaTester', 'roles_isInstructor', 'roles_isStaff', 'roles_isCCX', 'roles_isFinance', 'roles_isLibrary',
              'roles_isSales', 'forumRoles_isAdmin', 'forumRoles_isCommunityTA', 'forumRoles_isModerator', 
              'forumRoles_isStudent']
other_cols = ['mode', 'is_active', 'profile_country', 'y1_anomalous', 'email_domain']

In [96]:
for registration_col in registration_cols:
    print(registration_col)
    print(column_frequencies(original_records, registration_col))

registered
{'True': 1100005}
viewed
{'True': 568803, 'False': 531202}
explored
{'False': 450525, 'True': 118278, '': 531202}
certified
{'False': 1080771, 'True': 19234}
completed
{'False': 1076712, 'True': 23293}


Questions:
* Are there records where `certified` or `completed` are "True" but `explored` is "False" or Missing?

In [116]:
for demo_col in demo_cols:
    print(demo_col)
    freqs = column_frequencies(original_records, demo_col)
    print_sorted_col_by_key(freqs)
    print("")

LoE
 159905
a 49333
b 323862
el 4953
hs 234270
jhs 30482
m 227234
none 3340
other 20952
p 38673
p_oth 2794
p_se 4207

YoB
 167284
1893 7
1894 49
1895 58
1896 36
1897 79
1898 60
1899 48
1900 127
1901 8
1902 13
1903 1
1904 15
1906 17
1908 4
1909 18
1911 3
1914 7
1917 13
1918 23
1920 8
1921 2
1922 16
1923 28
1924 37
1925 15
1926 29
1927 15
1928 46
1929 103
1930 82
1931 171
1932 114
1933 175
1934 173
1935 156
1936 192
1937 289
1938 386
1939 447
1940 534
1941 592
1942 829
1943 800
1944 853
1945 829
1946 1432
1947 1603
1948 1848
1949 1685
1950 2076
1951 2264
1952 2007
1953 2498
1954 2638
1955 2865
1956 3015
1957 3619
1958 3344
1959 3840
1960 4782
1961 4095
1962 5047
1963 5244
1964 4901
1965 5867
1966 5901
1967 6520
1968 7663
1969 8264
1970 9366
1971 9417
1972 10101
1973 10113
1974 10901
1975 12178
1976 13498
1977 14990
1978 15742
1979 17283
1980 20633
1981 21318
1982 23088
1983 26641
1984 28479
1985 31868
1986 35181
1987 36568
1988 40677
1989 43797
1990 48874
1991 45994
1992 47820
1993 45917

Level of Education Questions:
*  For unreasonable or missing values, we can check if the same user supplied a more correct value in other records. 
*  Do we want to impute?

Year of Birth Questions:
* What's our cutoff for a reasonable value?
* For unreasonable or missing values, we can check if the same user supplied a more correct value in other records. 

Gender Questions
* Do we want to collapse "null" and missing? 
* Do we want to impute gender? (I'd rather not, given that it might not be missing randomly.
* For missing values, we can check if the same user supplied a more correct value in other records.

In [117]:
for grade_col in grade_cols:
    print(grade_col)
    freqs = column_frequencies(original_records, grade_col)
    print_sorted_col_by_key(freqs)
    print("")

grade
 344317
0.0 688412
0.01 4570
0.02 2931
0.03 3155
0.04 3082
0.05 2707
0.06 2192
0.07 1263
0.08 1982
0.09 2161
0.1 1407
0.11 1053
0.12 673
0.13 1067
0.14 1102
0.15 789
0.16 527
0.17 1029
0.18 658
0.19 396
0.2 829
0.21 394
0.22 511
0.23 562
0.24 295
0.25 615
0.26 557
0.27 328
0.28 275
0.29 549
0.3 455
0.31 312
0.32 197
0.33 449
0.34 240
0.35 374
0.36 215
0.37 355
0.38 281
0.39 317
0.4 328
0.41 193
0.42 265
0.43 281
0.44 191
0.45 248
0.46 155
0.47 186
0.48 109
0.49 85
0.5 244
0.51 143
0.52 272
0.53 184
0.54 96
0.55 147
0.56 206
0.57 209
0.58 167
0.59 128
0.6 194
0.61 224
0.62 251
0.63 171
0.64 295
0.65 300
0.66 141
0.67 283
0.68 170
0.69 193
0.7 1006
0.71 471
0.72 355
0.73 462
0.74 277
0.75 549
0.76 287
0.77 405
0.78 370
0.79 339
0.8 506
0.81 397
0.82 322
0.83 480
0.84 365
0.85 509
0.86 440
0.87 659
0.88 532
0.89 492
0.9 788
0.91 557
0.92 827
0.93 779
0.94 796
0.95 816
0.96 965
0.97 965
0.98 992
0.99 710
1.0 2926
1.01 1
1.11 5
1.15 13

passing_grade
 27866
0.5 70116
0.55 970
0.58 767

Grade Questions:
* Looks like extra credit might have existed?
* Do we want to convert to numeric? Or just put a note that whoever starts doing analysis would have to do that?

Passing Grade Questions:
* This column looks okay?

Time and Date Questions:
* How do we want to deal with checking these? Maybe RegEx to see that they follow the write date-time format?

In [123]:
for action_col in action_cols:
    print(action_col)
    freqs = column_frequencies(original_records, action_col)
    print_sorted_col_by_key(freqs)
    print("")

ndays_act
 255762
1 320058
10 11301
100 85
101 100
102 109
103 48
104 58
105 64
106 54
107 96
108 85
1083 1
109 37
11 9301
110 39
111 37
112 40
113 43
114 78
1146 6
115 52
116 48
1168 14
117 34
118 54
119 48
12 9568
120 67
121 46
122 49
123 33
124 30
125 38
126 45
127 19
128 60
129 29
13 7552
130 36
131 46
132 44
133 27
134 28
135 15
136 24
137 27
138 7
139 39
14 7001
140 7
141 40
142 36
143 27
144 26
145 33
146 35
147 24
148 40
149 9
1494 4
15 6482
150 23
1508 4
151 29
153 32
154 18
156 21
157 38
158 11
159 19
16 5551
160 12
162 12
163 12
164 13
165 14
166 38
167 31
168 6
169 16
17 4900
170 15
171 9
172 28
173 10
174 16
175 12
176 27
177 4
178 38
179 6
18 4174
180 39
181 9
184 10
1840 1
185 10
186 24
187 14
188 31
189 1
19 3947
190 37
191 5
1919 14
192 25
193 10
195 2
196 26
197 11
198 17
199 9
2 151227
20 3284
201 27
203 13
204 15
205 5
206 8
207 25
208 18
209 14
21 2972
210 2
212 6
213 13
215 19
216 12
217 22
218 27
22 3367
220 5
221 50
223 2
224 2
225 4
229 10
23 2720
230 14
235 10

40 1737
400 51
4001 15
401 23
402 54
403 26
404 65
4041 10
405 45
406 42
407 40
408 27
4085 5
409 21
4095 4
41 1419
410 63
4106 12
411 46
412 30
413 41
414 36
415 29
4158 9
416 82
4164 15
4169 12
417 47
418 18
419 58
4196 11
42 1718
420 59
421 31
422 28
4222 10
423 33
424 46
425 17
426 10
427 29
428 56
429 35
4293 6
43 1428
430 23
431 14
432 73
433 47
434 33
435 25
436 80
4368 6
437 22
438 12
439 36
44 1493
440 59
4401 1
441 72
4413 7
4418 2
442 54
443 33
444 28
4445 5
445 30
446 26
447 40
448 81
449 19
45 1642
450 47
4503 8
451 22
452 35
453 23
454 19
4548 10
455 68
456 6
457 10
458 27
459 45
4594 14
46 1461
460 34
4606 8
461 24
462 55
463 28
464 37
465 24
466 80
467 41
468 57
469 30
47 1199
470 59
4700 5
4702 9
471 42
472 31
473 14
474 25
475 41
47502 9
476 33
477 39
4779 12
478 16
479 66
47915 14
48 1573
480 55
481 72
482 38
483 23
484 30
4844 4
485 19
486 44
487 38
488 21
489 60
4890 7
49 1167
490 27
491 18
492 28
493 13
494 46
495 45
497 39
498 75
499 71
5 11368
50 1104
500 61
501

Action Questions:
* These look fine. 

In [98]:
for un_col in un_cols:
    print(un_col)
    print(column_frequencies(original_records, un_col))
    print("")

un_major_region
{'Northern America': 335033, 'South America': 64762, '': 185572, 'Central America': 21537, 'Caribbean': 5841, 'Western Europe': 52241, 'Southern Asia': 103877, 'Northern Europe': 58174, 'Western Asia': 29223, 'South-Eastern Asia': 39758, 'Eastern Asia': 48912, 'Northern Africa': 17740, 'Australia and New Zealand': 21395, 'Southern Europe': 41203, 'Eastern Europe': 39996, 'Eastern Africa': 10625, 'Southern Africa': 5857, 'Western Africa': 14402, 'Middle Africa': 1317, 'Central Asia': 1798, 'Melanesia': 540, 'Micronesia': 118, 'Polynesia': 84}
un_economic_group
{'Developed regions': 555685, 'Developing_Nations': 358748, '': 185572}
un_developing_nation
{'': 1083369, 'Least developed countries': 16636}
un_special_region
{'': 974667, 'Latin America and the Caribbean': 92140, 'Sub-Saharan-Africa': 33198}
